In [84]:
import pandas as pd, numpy as np, time
from sklearn.model_selection import train_test_split


In [86]:
%%time
flight= pd.read_csv('flights.csv')


Wall time: 58.7 s


In [87]:
flight.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
data = flight.sample(frac = 0.1, random_state=10)

In [89]:
data = data[["MONTH","DAY","DAY_OF_WEEK","AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT",
                 "ORIGIN_AIRPORT","AIR_TIME", "DEPARTURE_TIME","DISTANCE","ARRIVAL_DELAY"]]
data.dropna(inplace=True)


In [90]:
data.shape,flight.shape

((571339, 11), (5819079, 31))

In [91]:
data["ARRIVAL_DELAY"] = (data["ARRIVAL_DELAY"]>10)*1
data.head()

,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,DESTINATION_AIRPORT,ORIGIN_AIRPORT,AIR_TIME,DEPARTURE_TIME,DISTANCE,ARRIVAL_DELAY
411984,1,28,3,WN,103,MKE,DCA,102.0,713.0,634,0
3591965,8,11,2,B6,153,PBI,JFK,134.0,111.0,1028,1
526451,2,4,3,DL,1187,DCA,MSP,111.0,1734.0,931,0
1336011,3,27,5,WN,171,RDU,DEN,173.0,1807.0,1436,0
3424502,8,1,6,WN,4330,RIC,ATL,63.0,2151.0,481,1


In [92]:
data["AIRLINE"].astype("category").cat.codes.unique()

array([13,  2,  3,  4,  6,  0,  7,  9, 10, 12,  8, 11,  1,  5],
      dtype=int64)

In [93]:
cols = ["AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT","ORIGIN_AIRPORT"]
for item in cols:
    data[item] = data[item].astype("category").cat.codes+1
 
train, test, y_train, y_test = train_test_split(data.drop(["ARRIVAL_DELAY"], axis=1), data["ARRIVAL_DELAY"],
                                                random_state=10, test_size=0.25)


In [94]:
import lightgbm as lgb
from sklearn import metrics
from sklearn.model_selection import GridSearchCV

In [95]:
y_train[y_train==1].count()/y_train[y_train==0].count()


0.2806493743256854

In [128]:
def auc2(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict(train)),
                            metrics.roc_auc_score(y_test,m.predict(test)))
def auc_pr(m,train,test):
    precision_tr, recall_tr, thresholds_tr = metrics.precision_recall_curve(y_train,m.predict(train))
    precision_te, recall_te, thresholds_te = metrics.precision_recall_curve(y_test,m.predict(test))
    auc_tr = metrics.auc(recall_tr, precision_tr)
    auc_te = metrics.auc(recall_te, precision_te)
  #  f1_tr=metrics.f1_score(y_train,m.predict(train))
   # f1_te=metrics.f1_score(y_test,m.predict(test))
    return(print('auc_tr=%.3f' % auc_tr),print('auc_te=%.3f' % auc_te))

#params = {'boosting_type': 'gbdt',
#          'objective': 'binary',
#         'metric':'precision',
#         'is_unbalance':'True',
 #        'silent' : 'True'}

lg = lgb.LGBMClassifier(objective= 'binary',
         boosting_type= 'gbdt',
        n_jobs = 5, 
          silent = True,
         is_unbalance=True
    #    early_stopping_round=10,
     #   metric='precision_score'
                        
         )

param_grid = {"max_depth": [6, 15,30],
              "learning_rate" : [0.05,0.1],
              'num_iterations': [3000],
               'num_leaves': [ 27,70, 100],
             'random_state' : [501],
              'feature_fraction':[0.5,0.7,1],
              'bagging_fraction':[0.9],
             'lambda_l1':[0.1], 
              'lambda_l2':[0.1], 
              'min_split_gain':[0.01],
              'min_child_weight':[40]
             }

In [ ]:
%%time
#grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 5, scoring="roc_auc", verbose=5)
grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_grid, cv = 3, scoring="precision", verbose=1)
grid_search.fit(train,y_train)
#grid_search.best_estimator_
print ('Best score: %0.3f' % grid_search.best_score_)
    


Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 19.5min


In [99]:
print(grid_search.best_params_)

{'feature_fraction': 0.7, 'learning_rate': 0.1, 'max_depth': 30, 'n_estimators': 200, 'num_leaves': 100, 'random_state': 501}


In [101]:
d_train = lgb.Dataset(train, label=y_train,free_raw_data=False)
params = {"max_depth": 15, "learning_rate" : 0.1, "num_leaves": 100, 'objective':'binary','num_iterations':3000,
         'random_state':501,'feature_fraction': 0.7,'early_stopping_round':10,'metric':'precision_score'}

# Without Categorical Features
#model2 = lgb.train(params, d_train)
#auc2(model2, train, test)

cate_features_name = ["MONTH","DAY","DAY_OF_WEEK","AIRLINE","DESTINATION_AIRPORT","ORIGIN_AIRPORT"]
model2 = lgb.train(params, d_train, categorical_feature = cate_features_name)

In [102]:
### Predict train
#predictions = grid_search.predict(test)
predictions_lgbm_prob=model2.predict(train)
                              #       ,pred_contrib=False)
train['prob']= predictions_lgbm_prob


predictions_lgbm_labeled = np.where(predictions_lgbm_prob > 0.3, 1, 0)



print(y_train[y_train==1].count()/y_train[y_train==0].count())



print(metrics.classification_report(y_train, predictions_lgbm_labeled))

#print(metrics.confusion_matrix(y_train, predictions_lgbm_labeled,labels=[y_train.unique()[0],y_train.unique()[1]]))

##########################

0.2806493743256854
              precision    recall  f1-score   support

           0       0.90      0.88      0.89    334599
           1       0.60      0.63      0.62     93905

    accuracy                           0.83    428504
   macro avg       0.75      0.76      0.75    428504
weighted avg       0.83      0.83      0.83    428504



In [103]:
### Predict test
#predictions = grid_search.predict(test)
predictions_lgbm_prob=model2.predict(test)

predictions_lgbm_labeled = np.where(predictions_lgbm_prob > 0.3, 1, 0)


print(y_test[y_test==1].count()/y_test[y_test==0].count())


print(metrics.classification_report(y_test, predictions_lgbm_labeled))

print(metrics.confusion_matrix(y_test, predictions_lgbm_labeled))

##########################





0.2794707800351141
              precision    recall  f1-score   support

           0       0.86      0.85      0.86    111636
           1       0.49      0.50      0.50     31199

    accuracy                           0.78    142835
   macro avg       0.67      0.68      0.68    142835
weighted avg       0.78      0.78      0.78    142835

[[95190 16446]
 [15448 15751]]


346452

In [33]:
auc_pr(model2, train, test)


auc_tr=0.455
auc_te=0.445


(None, None)